In [13]:
import pandas as pd
import numpy as np
import os
import seaborn as sns


In [2]:
caminho = 'Dados'

In [5]:
clientes = pd.read_csv(os.path.join(caminho, 'olist_customers_dataset.csv'))
pedidos = pd.read_csv(os.path.join(caminho, 'olist_orders_dataset.csv'))
itens_pedido = pd.read_csv(os.path.join(caminho, 'olist_order_items_dataset.csv'))
produtos = pd.read_csv(os.path.join(caminho, 'olist_products_dataset.csv'))
vendedores = pd.read_csv(os.path.join(caminho, 'olist_sellers_dataset.csv'))
pagamentos = pd.read_csv(os.path.join(caminho, 'olist_order_payments_dataset.csv'))
reviews = pd.read_csv(os.path.join(caminho, 'olist_order_reviews_dataset.csv'))
categorias = pd.read_csv(os.path.join(caminho, 'product_category_name_translation.csv'))

In [6]:
datasets = {
    'clientes': clientes,
    'pedidos': pedidos,
    'itens_pedido': itens_pedido,
    'produtos': produtos,
    'vendedores': vendedores,
    'pagamentos': pagamentos,
    'reviews': reviews
}

for nome, df in datasets.items():
    print(f'--- {nome.upper()} ---')
    print(df.isnull().sum())
    print()

--- CLIENTES ---
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

--- PEDIDOS ---
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

--- ITENS_PEDIDO ---
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

--- PRODUTOS ---
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm   

In [7]:
# Preencher ou remover nulos
produtos['product_category_name'].fillna('unknown', inplace=True)

# Remover duplicatas se houver
for nome, df in datasets.items():
    datasets[nome] = df.drop_duplicates()


C:\Users\mateu\AppData\Local\Temp\ipykernel_3148\3396239352.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  produtos['product_category_name'].fillna('unknown', inplace=True)


In [8]:
pedidos['order_purchase_timestamp'] = pd.to_datetime(pedidos['order_purchase_timestamp'])
pedidos['order_approved_at'] = pd.to_datetime(pedidos['order_approved_at'])
pedidos['order_delivered_carrier_date'] = pd.to_datetime(pedidos['order_delivered_carrier_date'])
pedidos['order_delivered_customer_date'] = pd.to_datetime(pedidos['order_delivered_customer_date'])
pedidos['order_estimated_delivery_date'] = pd.to_datetime(pedidos['order_estimated_delivery_date'])


In [9]:
# Merge: pedidos + clientes
df = pedidos.merge(clientes, on='customer_id', how='left')

# Merge: adicionar itens do pedido
df = df.merge(itens_pedido, on='order_id', how='left')

# Merge: produtos
df = df.merge(produtos, on='product_id', how='left')

# Merge: categorias traduzidas
df = df.merge(categorias, on='product_category_name', how='left')

# Merge: pagamentos
df = df.merge(pagamentos, on='order_id', how='left')

# Merge: avaliações
df = df.merge(reviews, on='order_id', how='left')


In [10]:
df.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,1.0,credit_card,1.0,18.12,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,3.0,voucher,1.0,2.00,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,2.0,voucher,1.0,18.59,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,47813,...,1.0,boleto,1.0,141.46,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,1.0,credit_card,3.0,179.12,e73b67b67587f7644d5bd1a52deb1b01,5.0,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58


In [11]:
df.to_csv('Dados/dataset_olist_preparado.csv', index=False)


In [14]:
sns.set(style="whitegrid")

# Carregar dataset preparado
df = pd.read_csv('Dados/dataset_olist_preparado.csv', parse_dates=['order_purchase_timestamp'])